In [1]:
# install or import only if you want to calculate pCO2
# pip install PyCO2SYS
# import PyCO2SYS as pyco2

In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import numpy.ma as ma
%matplotlib inline
%config InlineBackend.figure_format = 'jpg'
%config InlineBackend.print_figure_kwargs = {'dpi':300, 'bbox_inches': 'tight'}
import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
import matplotlib.pyplot as plt
import scipy
import sklearn.linear_model 
import pickle
import re
import requests
from datetime import datetime
import tempfile

In [3]:
import gcsfs 
fs = gcsfs.GCSFileSystem()

In [4]:
# From: https://hahana.soest.hawaii.edu/hot/hotco2/hotco2.html
# get .txt file: https://hahana.soest.hawaii.edu/hot/hotco2/HOT_surface_CO2.txt

In [25]:
fs.ls('gs://leap-persistent/ckg-2dxsu/Taylor_data/databases/raw')

['leap-persistent/ckg-2dxsu/Taylor_data/databases/raw/GLODAPv2.2023_Merged_Master_File.csv.zip',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/raw/HOT_surface_CO2.txt',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/raw/LDEO_Database_V2019.csv',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/raw/bats_bottle.txt']

In [6]:
# In this part, we need to set the download path and save path.
# And the script would download the data and save it directly to GCS.
# If failed, the output would notify

# Set the path of HOT data.
# Available here: https://hahana.soest.hawaii.edu/hot/hotco2/hotco2.html
# Current version information will be shown in the next cell's output
hot_url = 'https://hahana.soest.hawaii.edu/hot/hotco2/HOT_surface_CO2.txt'

# GCS path to save the data
save_path = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/databases'
# Edit the file name if needed.
gcs_path = f'{save_path}/raw/HOT_surface_CO2.txt'

fs = gcsfs.GCSFileSystem()

if fs.exists(gcs_path):
    print(f"File already exists at {gcs_path}. Skipping download.")
else:
    # download and save to GCS
    response = requests.get(hot_url, stream=True)
    if response.status_code == 200:
        with fs.open(gcs_path, 'wb') as gcs_file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive new chunks
                    gcs_file.write(chunk)
        print(f"Succesfully downloaded and saved to {gcs_path}")
    else:
        print(f"Error! Could not download: {response.status_code}")

Succesfully downloaded and saved to gs://leap-persistent/ckg-2dxsu/Taylor_data/databases/raw/HOT_surface_CO2.txt


In [7]:
# Load data from GCS and convert to dataframe
with fs.open(gcs_path, 'rb') as gcs_file:
    lines = gcs_file.readlines()

# Mannully set where data starts from, skipping the comment lines. Here we use: 9
# Shound not change if HOT keeps their format
lines = [line.decode('utf-8') for line in lines]
data = lines[8:]

# Get column names
columns = data[0].replace(',', '').strip().split()

df = pd.DataFrame([line.strip().split() for line in data[1:]], columns=columns)

# Detect the latest cruise date and convert to yyyy-mm-dd
# As long as HOT keeps the same format, this code would work.
# Current format: The 4th line: Last updated 11 December 2023 by J.E. Dore
info_line = lines[3]
date_pattern = r'\b\d{1,2}\s+[A-Za-z]+\s+\d{4}\b'
match = re.search(date_pattern, info_line)
if match:
    date_str = match.group(0)
    date_obj = datetime.strptime(date_str, '%d %B %Y')
    formatted_month = date_obj.strftime('%Y%m')
    formatted_date = date_obj.strftime('%Y-%m-%d')
    print(f"Last updated on: {formatted_date}")
else:
    raise ValueError("Take care. Fail to detect the latest cruise date!")

Last updated on: 2023-12-11


In [8]:
df.columns

Index(['cruise', 'days', 'date', 'temp', 'sal', 'phos', 'sil', 'DIC', 'TA',
       'nDIC', 'nTA', 'pHmeas_25C', 'pHmeas_insitu', 'pHcalc_25C',
       'pHcalc_insitu', 'pCO2calc_insitu', 'pCO2calc_20C',
       'aragsatcalc_insitu', 'calcsatcalc_insitu', 'freeCO2_insitu',
       'carbonate_insitu', 'notes'],
      dtype='object')

In [9]:
df = df[['days', 'date', 'temp', 'pCO2calc_insitu','sal','phos','sil','DIC','TA']]
# df=df.drop(columns=['cruise','nDIC','nTA', 'pHmeas_25C', 'pHmeas_insitu','pHcalc_25C','pHcalc_insitu','pCO2calc_20C','aragsatcalc_insitu','calcsatcalc_insitu','freeCO2_insitu','carbonate_insitu','notes'])

In [10]:
df.head()

,days,date,temp,pCO2calc_insitu,sal,phos,sil,DIC,TA
0,30,31-Oct-88,26.283,330.9,35.186,0.08,0.71,1963.91,2319.5
1,62,2-Dec-88,25.659,330.6,34.984,0.09,0.99,1958.94,2304.9
2,99,8-Jan-89,24.61,324.3,35.028,0.07,0.93,1963.77,2305
3,148,26-Feb-89,23.479,310.9,34.883,0.09,0.88,1957.8,2295.5
4,177,27-Mar-89,24.278,317.7,34.735,0.12,2.01,1946.33,2283


In [11]:
# Convert data types
columns_to_convert = ['days','temp','pCO2calc_insitu']
# columns_to_convert = ['days','temp', 'sal', 'phos', 'sil', 'DIC', 'TA']


for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

df.head()

,days,date,temp,pCO2calc_insitu,sal,phos,sil,DIC,TA
0,30,31-Oct-88,26.283,330.9,35.186,0.08,0.71,1963.91,2319.5
1,62,2-Dec-88,25.659,330.6,34.984,0.09,0.99,1958.94,2304.9
2,99,8-Jan-89,24.610,324.3,35.028,0.07,0.93,1963.77,2305
3,148,26-Feb-89,23.479,310.9,34.883,0.09,0.88,1957.8,2295.5
4,177,27-Mar-89,24.278,317.7,34.735,0.12,2.01,1946.33,2283


In [12]:
# Grab observations
# drop those columns where pCO2calc_insitu values are missing
ds = df[(df.pCO2calc_insitu != -999)]
# ds = df[(df.TA > 0) & (df.sal > 0) & (df.temp != -999) & (df.DIC > 0) & (df.sil != -999) & (df.phos != -999)]
ds.info

<bound method DataFrame.info of       days       date    temp  pCO2calc_insitu     sal  phos   sil      DIC  \
0       30  31-Oct-88  26.283            330.9  35.186  0.08  0.71  1963.91   
1       62   2-Dec-88  25.659            330.6  34.984  0.09  0.99  1958.94   
2       99   8-Jan-89  24.610            324.3  35.028  0.07  0.93  1963.77   
3      148  26-Feb-89  23.479            310.9  34.883  0.09  0.88   1957.8   
4      177  27-Mar-89  24.278            317.7  34.735  0.12  2.01  1946.33   
..     ...        ...     ...              ...     ...   ...   ...      ...   
334  12231  28-Mar-22  24.497            390.5  35.094  0.04  1.12  2005.70   
335  12291  27-May-22  24.892            404.3  34.753  0.10  1.22  1996.00   
336  12335  10-Jul-22  25.344            396.2  35.255  0.04  1.17  2017.40   
337  12356  31-Jul-22  25.969            409.0  35.312  0.02  1.29  2016.55   
338  12388  01-Sep-22  26.659            408.8  35.005  0.05  1.16  1995.35   

         TA  
0    

In [13]:
# The following 3 cells are using PyCO2SYS package to calculate pCO2. 
# But we would not use this result for now. 
# If you want to use this pCO2 value, search for "'spco2':(['time'],CO2dict['pCO2'])},". Use that hot_out to save as netCDF to cloud.

# Define input conditions
# These are the inputs to calculate pCO2 using PyCO2SYS package
par1type =    1  # The first parameter supplied is of type "1", which is "alkalinity"
par1     = ds.TA  # Value of the first parameter
par2type =    2  # The second parameter supplied is of type "2", which is "DIC"
par2     = ds.DIC  # Value of the second parameter
sal      = ds.sal  # Salinity of the sample
tempin   = ds.temp  # Temperature at input conditions
presin   = 0  # Pressure    at input conditions
sil      = ds.sil #50  # Concentration of silicate  in the sample (in umol/kg)
po4      = ds.phos  # 2# Concentration of phosphate in the sample (in umol/kg)
pHscale  =    1  # pH scale at which the input pH is reported ("1" means "Total Scale")  
                 #  - doesn't matter in this example
k1k2c    =    10 #4  # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("4" means "Mehrbach refit")  (Galen says use "10")
kso4c    =    1  # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")

In [14]:
# Run CO2SYS!
'''
CO2dict = pyco2.sys(par1, par2, par1type, par2type,
                    salinity=sal, temperature=tempin, pressure=presin,
                    total_silicate=sil, total_phosphate=po4,
                    opt_pH_scale=pHscale, opt_k_carbonic=k1k2c, opt_k_bisulfate=kso4c)
print('PyCO2SYS ran successfully!')
'''

"\nCO2dict = pyco2.sys(par1, par2, par1type, par2type,\n                    salinity=sal, temperature=tempin, pressure=presin,\n                    total_silicate=sil, total_phosphate=po4,\n                    opt_pH_scale=pHscale, opt_k_carbonic=k1k2c, opt_k_bisulfate=kso4c)\nprint('PyCO2SYS ran successfully!')\n"

In [15]:
'''
fig = plt.subplots(1,1,figsize=(12,2))
plt.scatter(ds.days,CO2dict['pCO2'])
plt.xlim(min(ds.days),max(ds.days))
'''

"\nfig = plt.subplots(1,1,figsize=(12,2))\nplt.scatter(ds.days,CO2dict['pCO2'])\nplt.xlim(min(ds.days),max(ds.days))\n"

In [16]:
ds.columns

Index(['days', 'date', 'temp', 'pCO2calc_insitu', 'sal', 'phos', 'sil', 'DIC',
       'TA'],
      dtype='object')

In [17]:
ds['Date']=ds['date'].apply(lambda x: datetime.strptime(x, '%d-%b-%y').strftime('%Y-%m-%d'))


/tmp/ipykernel_1356/78431236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['Date']=ds['date'].apply(lambda x: datetime.strptime(x, '%d-%b-%y').strftime('%Y-%m-%d'))


In [18]:
ds.head()

,days,date,temp,pCO2calc_insitu,sal,phos,sil,DIC,TA,Date
0,30,31-Oct-88,26.283,330.9,35.186,0.08,0.71,1963.91,2319.5,1988-10-31
1,62,2-Dec-88,25.659,330.6,34.984,0.09,0.99,1958.94,2304.9,1988-12-02
2,99,8-Jan-89,24.610,324.3,35.028,0.07,0.93,1963.77,2305,1989-01-08
3,148,26-Feb-89,23.479,310.9,34.883,0.09,0.88,1957.8,2295.5,1989-02-26
4,177,27-Mar-89,24.278,317.7,34.735,0.12,2.01,1946.33,2283,1989-03-27


In [19]:
has_invalid_values = (ds == -999).any().any()
has_invalid_values

False

In [20]:
# If the output of previous cell is "True", which means that there are -999 value, then run this cell

# ds.replace(-999, np.nan, inplace=True)
# has_invalid_values = (ds == -999).any().any()
# has_invalid_values

In [21]:
hot_out = xr.Dataset({
                        'temp':(["time"],ds['temp']),                       
                        'spco2':(['time'],ds['pCO2calc_insitu'])},
                        coords={'time': (['time'],ds['Date'])})

In [22]:
# Use this hot_out only if you want to use the calculated pCO2 value by PyCO2SYS package.
'''
hot_out = xr.Dataset({
                        'temp':(["time"],ds['temp']),
                        'salinity':(['time'],ds['sal']),
                        'CO2':(['time'],ds['DIC']),
                        'alk':(['time'],ds['TA']),
                        'Si':(['time'],ds['sil']),
                        'PO4':(['time'],ds['phos']),
                        'spco2':(['time'],CO2dict['pCO2'])},
                        coords={'time': (['time'],ds['Date'])})
'''

'\nhot_out = xr.Dataset({\n                        \'temp\':(["time"],ds[\'temp\']),\n                        \'salinity\':([\'time\'],ds[\'sal\']),\n                        \'CO2\':([\'time\'],ds[\'DIC\']),\n                        \'alk\':([\'time\'],ds[\'TA\']),\n                        \'Si\':([\'time\'],ds[\'sil\']),\n                        \'PO4\':([\'time\'],ds[\'phos\']),\n                        \'spco2\':([\'time\'],CO2dict[\'pCO2\'])},\n                        coords={\'time\': ([\'time\'],ds[\'Date\'])})\n'

In [23]:
# Save to GCS as zarr
zarr_gcs_path = f'{save_path}/processed/HOT_spco2_{formatted_month}.zarr'

'''with tempfile.NamedTemporaryFile(suffix='.nc') as tmp_file:
    hot_out.to_netcdf(tmp_file.name)
    tmp_file.seek(0) 
    with fs.open(zarr_gcs_path, 'wb') as gcs_file:
        gcs_file.write(tmp_file.read())'''

hot_out.to_zarr( zarr_gcs_path, mode='w')

# Check if saved sucessfully
if fs.exists(zarr_gcs_path):
    print(f"Successfully saved to {zarr_gcs_path}")
else:
    print(f"Failed to save to {zarr_gcs_path}")

Successfully saved to gs://leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/HOT_spco2_202312.zarr


In [24]:
fs.ls(save_path + '/processed')

['leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/HOT_spco2_202312.zarr',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/bats_spco2_1988-10-2023-06.zarr']

In [ ]:
# If you want to delete file from the GCS path, run the following code
'''
delete_path = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/GLODAPv2.2023_Merged_Master_File.csv'

fs = gcsfs.GCSFileSystem()

if fs.exists(delete_path):
    fs.rm(delete_path, recursive=True)
    print(f"The file {delete_path} has been deleted")
else:
    print(f"The file {delete_path} does not exist")
'''